In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm



 # Training a Classifier

 ## Training an image classifier

 We will do the following steps in order:

 1. Load and normalize the CIFAR10 training and test datasets using
    ``torchvision``
 2. Define a Convolutional Neural Network
 3. Define a loss function
 4. Train the network on the training data
 5. Test the network on the test data

 ### 1. Load and normalize CIFAR10

 Using ``torchvision``, it’s extremely easy to load CIFAR10.


 The output of torchvision datasets are PILImage images of range [0, 1].
 We transform them to Tensors of normalized range [-1, 1].



In [ ]:
data_dir = Path.cwd().parent.joinpath("data", "image-classification")
model_dir = Path.cwd().parent.joinpath("models", "image-classification")
assert data_dir.exists(), "Data directory not found!"
assert model_dir.exists(), "Model directory not found!"
SAVEPATH = model_dir.joinpath("cifar_net.pth")


In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ]
)

batch_size = 4

trainset = torchvision.datasets.CIFAR10(
    root=data_dir, train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=0
)

testset = torchvision.datasets.CIFAR10(
    root=data_dir, train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=0
)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)



 Let us show some of the training images, for fun.


In [ ]:

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)


In [ ]:
# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(" ".join(f"{classes[labels[j]]:5s}" for j in range(batch_size)))



 ### 2. Define a Convolutional Neural Network
 Copy the neural network from the Neural Networks section before and modify it to
 take 3-channel images (instead of 1-channel images as it was defined).



In [ ]:


class Net(nn.Module):  # Python subclass of nn.Module
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()



 ### 3. Define a Loss function and optimizer
 Let's use a Classification Cross-Entropy loss and SGD with momentum.



In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



 ### 4. Train the network

 This is when things start to get interesting.
 We simply have to loop over our data iterator, and feed the inputs to the
 network and optimize.



In [ ]:

for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    # Add a progress bar over the training loop
    with tqdm(enumerate(trainloader, 0), total=len(trainloader)) as pbar:
        # for i, data in enumerate(trainloader, 0):
        pbar.set_description(f"[epoch={epoch + 1}, minibatch={0}] loss: [running...]")
        # for i, data in enumerate(trainloader, 0):
        for i, data in pbar:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                pbar.set_description(
                    f"[epoch={epoch + 1}, minibatch={i + 1:5d}] loss: {running_loss / 2000:.3f}"
                )
                running_loss = 0.0

print("Finished Training")



 Let's quickly save our trained model:



In [ ]:
torch.save(net.state_dict(), SAVEPATH)



 See [here](https://pytorch.org/docs/stable/notes/serialization.html)
 for more details on saving PyTorch models.

 ### 5. Test the network on the test data

 We have trained the network for 2 passes over the training dataset.
 But we need to check if the network has learnt anything at all.

 We will check this by predicting the class label that the neural network
 outputs, and checking it against the ground-truth. If the prediction is
 correct, we add the sample to the list of correct predictions.

 Okay, first step. Let us display an image from the test set to get familiar.



In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print("GroundTruth: ", " ".join(f"{classes[labels[j]]:5s}" for j in range(4)))



 Next, let's load back in our saved model (note: saving and re-loading the model
 wasn't necessary here, we only did it to illustrate how to do so):



In [ ]:
net = Net()
net.load_state_dict(torch.load(SAVEPATH))



 Okay, now let us see what the neural network thinks these examples above are:



In [ ]:
outputs = net(images)



 The outputs are energies for the 10 classes.
 The higher the energy for a class, the more the network
 thinks that the image is of the particular class.
 So, let's get the index of the highest energy:



In [ ]:
_, predicted = torch.max(outputs, 1)

print("Predicted: ", " ".join(f"{classes[predicted[j]]:5s}" for j in range(4)))



 The results seem pretty good.

 Let us look at how the network performs on the whole dataset.



In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct // total} %")



 That looks way better than chance, which is 10% accuracy (randomly picking
 a class out of 10 classes).
 Seems like the network learnt something.

 Hmmm, what are the classes that performed well, and the classes that did
 not perform well:



In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f"Accuracy for class: {classname:5s} is {accuracy:.1f} %")



 > Note: See
 > [Source](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) for
 > information on running with GPU.